# CS 447 Homework 1 $-$ Language Models
In this homework we will study some traditional appraoches to language modeling: in particular, you will build some n-gram language models on a corpus of Wikipedia articles.

This notebook is designed to be run in Google Colab. Navigate to <TT>colab.research.google.com</TT> and upload this notebook. Then follow the instructions in the notebook to do the assignent.

To run the notebook, you will need to connect to a Runtime. For this homework, all you need is a CPU. You can change the runtime by going to <TT>Runtime > Change runtime type</TT> and selecting <TT>None</TT> in the <TT>Hardware Accelerator</TT> field. We encourage you to disconnect from the runtime when you are not using it, as Google Colab can limit your resources if you overuse them.

You can read more about Google Colab at https://research.google.com/colaboratory/faq.html.

We have imported all the libraries you need to do this homework. <b>You should not import any extra libraries.</b> If you do, the autograder will fail to run your code.

Here, you will train some <b>n-gram language models</b> on WikiText-2, a corpus of high-quality Wikipedia articles. The dataset was originally introduced in the following paper: https://arxiv.org/pdf/1609.07843v1.pdf. A raw version of the data can easily be viewed here: https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2.

## Preprocessing the Data

To make your models more robust, it is necessary to perform some basic preprocessing on the corpora. <i>You do not need to edit this code.</i>

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp;In this homework, we are interested in modeling individual sentences, rather than longer chunks of text such as paragraphs or documents. The WikiTest dataset provides paragraphs; thus, we provide a simple method to identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`). These markers will allow your models to generate sentences that have realistic beginnings and endings.

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;In order to deal with unknown words in the test corpora, all words that do not appear in the vocabulary must be replaced with a special token for unknown words (`<UNK>`) before estimating your models. The WikiText dataset has already done this, and you can read about the method in the paper above. When unknown words are encountered in the test corpus, they should be treated as that special token instead.

We provide you with preprocessing code here, and you should not modify it.

After the preprocessing, you may assume that all words in the test set appear in the training set, as this code has already replaced the unseen tokens with `<UNK>`.

In [ ]:
# Constants (feel free to use these in your code, but do not change them)
START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [ ]:
### DO NOT EDIT ###
import os
import random
import sys
from urllib.request import urlretrieve

def preprocess(data, vocab=None):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = set()
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    splits = ['train', 'valid']
    datasets = []
    path = './{}.txt'
    url = 'https://raw.githubusercontent.com/pytorch/examples/main/word_language_model/data/wikitext-2/{}.txt'
    for split in splits:
        if os.path.exists(path.format(split)):
            print(f"{split} dataset already downloaded")
        else:
            filename = f'{split}.txt'
            urlretrieve(url.format(split), filename)
        datasets.append(open(f'{split}.txt').read().split('\n'))
    train_dataset, vocab = preprocess(datasets[0])
    test_dataset, _ = preprocess(datasets[1], vocab)
    return train_dataset, test_dataset

if __name__ == '__main__':
    train_dataset, test_dataset = getDataset()

Run the next cell to see 10 random sentences of the training data.

In [ ]:
if __name__ == '__main__':
    for x in random.sample(train_dataset, 10):
        print (x)

## The LanguageModel Class

You will implement 4 types of language models: a <b>unigram</b> model, a <b>smoothed unigram</b> model, a <b>bigram</b> model, and a <b>smoothed bigram</b> model. Each of the models is worth 25 points and extends the following base class. <b>You do not need to implement anything in this class</b>; you will instead implement each of the following methods in the relevant subclass:

* <b>`__init__(self, trainCorpus)`</b>: Train the language model on `trainCorpus`. This will involve calculating relative frequency estimates according to the type of model you're implementing.

* <b>`generateSentence(self)`</b>: <b>[5 points]</b> Return a sentence that is generated by the language model. It should be a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>, where each <TT>w<sup>(i)</sup></TT> is a word in your vocabulary (including <TT>&lt;UNK&gt;</TT> but exlcuding <TT>&lt;s&gt;</TT> and <TT>&lt;&sol;s&gt;</TT>). You may assume that <TT>&lt;s&gt;</TT> starts each sentence (with probability $1$). The following words <TT>w<sup>(1)</sup></TT>, ... , <TT>w<sup>(n)</sup></TT>, <TT>&lt;&sol;s&gt;</TT> are generated according to your language model's distribution. Note that the number of words <TT>n</TT> is not fixed; instead, you should stop the sentence as soon as you generate the stop token <TT>&lt;&sol;s&gt;</TT>.

* <b>`getSentenceLogProbability(self, sentence)`</b>: <b>[10 points]</b> Return the <em> logarithm of the probability</em> of <TT>sentence</TT>, which is again a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>. You should use the natural logarithm $-$ that is, the base-<em>e</em> logarithm. See the note below about performing your calculations in log space.

* <b>`getCorpusPerplexity(self, testCorpus)`</b>: <b>[10 points]</b> You need to compute the perplexity (normalized inverse log probability) of `testCorpus` according to your model. For a corpus $W$ with $N$ words and a bigram model, Jurafsky and Martin tells you to compute perplexity as follows:

$$Perplexity(W) = \Big [ \prod_{i=1}^N \frac{1}{P(w^{(i)}|w^{(i-1)})} \Big ]^{1/N}$$

<b>Implementation Hint:</b> In order to avoid underflow, you will likely need to do all of your calculations in log-space. That is, instead of multiplying probabilities, you should add the logarithms of the probabilities and exponentiate the result:

$$\prod_{i=1}^N P(w^{(i)}|w^{(i-1)}) = \exp\Big (\sum_{i=1}^N \log P(w^{(i)}|w^{(i-1)}) \Big ) $$

Using this property should help you in your implementation of `generateSentence(self)` and `getCorpusPerplexity(self, testCorpus)`.

Feel free to implement helper methods as you wish (either in the base class or in the subclases). <b>But be sure not to change the function signatures of the provided methods</b> (i.e. the function and argument names), or else the autograder will fail.

In [ ]:
import math
import random
from collections import defaultdict

class LanguageModel(object):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model (i.e. estimate the model's underlying probability
        distribution from the training corpus.)
        '''

        ### DO NOT EDIT ###
        return

    def generateSentence(self):
        '''
        Generate a sentence by drawing words according to the model's probability distribution.
        Note: Think about how to set the length of the sentence in a principled way.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement generateSentence in each subclass.")

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the log probability of the sentence provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getSentenceProbability in each subclass.")

    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexity of the corpus provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getCorpusPerplexity in each subclass.")

    def printSentences(self, n):
        '''
        Prints n sentences generated by your model.
        '''

        ### DO NOT EDIT ###
        for i in range(n):
            sent = self.generateSentence()
            prob = self.getSentenceLogProbability(sent)
            print('Log Probability:', prob , '\tSentence:',sent)

## <font color='red'>TODO:</font> Unigram Model [25 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed unigram</b> model. The probability distribution of a word is given by $\hat P(w)$.

<font color='green'><b>Hints:</b></font>
* <font color='green'>You should use a <b>dictionary</b> to map tokens to their unigram counts.</font>
* <font color='green'>Since you never want to generate the start-of-sentence token `<s>`, you should <b>not</b> include it in your counts.</font>
* <font color='green'>In general, avoid checking for membership in a list (i.e. avoid `x in lst`). Instead, use sets or dictionaries for this purpose $-$ membership checks are much faster on these data structures.</font>
* <font color='green'>Do <b>not</b> modify the training or test corpora by using `.append(...)` or `.pop(...)` on them. This will cause unexpected behavior in the autograder tests, which do not expect you to be changing the data.

In [ ]:
class UnigramModel(LanguageModel):
    def __init__(self, trainCorpus):

        ### TODO ###
        pass

    def generateSentence(self):

        ### TODO ###
        return []

    def getSentenceLogProbability(self, sentence):

        ### TODO ###
        return 0.0

    def getCorpusPerplexity(self, testCorpus):

        ### TODO ###
        return 0.0

We provide you with a testing function that uses very simple training & test corpora (you could compute probability/perplexity by hand if you wanted to). This is just a <b>sanity check</b> $-$ passing this test does not guarantee you a perfect score in the autograder; this is simply to help you debug your model.

In [ ]:
def sanityCheck(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}

    #	Read in the test corpus
    train_corpus = ["By the Late Classic , a network of few <unk> ( few <unk> ) linked various parts of the city , running for several kilometres through its urban core .",
    "Few people realize how difficult it was to create Sonic 's graphics engine , which allowed for the incredible rate of speed the game 's known for ."]
    test_corpus = ["Classic few parts of the game allowed for few <unk> <unk> incredible city .",
                   "Few <unk> realize the difficult network , which linked the game to Sonic ."]
    train_corpus, _ = preprocess(train_corpus)
    test_corpus, _ = preprocess(test_corpus)
    sentence = preprocess(["Sonic was difficult ."])[0][0]

    # These are the correct answers (don't change them!)
    if model_type == "unigram":
       senprobs = [-19.08542845, -114.5001481799, -108.7963657053, -53.6727664115, -55.4645258807]
       trainPerp, testPerp = 41.3308239726, 38.0122981569
       model = UnigramModel(train_corpus)
    elif model_type == "smoothed-unigram":
       senprobs = [-19.0405293515, -115.3479413049, -108.9114348746, -54.8190029616, -55.8122547346]
       trainPerp, testPerp = 41.9994393615, 39.9531928383
       model = SmoothedUnigramModel(train_corpus)
    elif model_type == "bigram":
       senprobs = [-float('inf'), -10.3450917073, -9.2464794186, -float('inf'), -float('inf')]
       trainPerp, testPerp = 1.3861445461, float('inf')
       model = BigramModel(train_corpus)
    elif model_type == "smoothed-bigram":
       senprobs = [-16.355820202, -76.0026113319, -74.2346475108, -47.2885760372, -51.2730261907]
       trainPerp, testPerp = 12.2307627397, 26.7193157699
       model = SmoothedBigramModelAD(train_corpus)
    else: assert False, 'Invalid model_type'

    print("--- TEST: generateSentence() ---")
    modelSen = model.generateSentence()
    senTestPassed = isinstance(modelSen, list) and len(modelSen) > 1 and isinstance(modelSen[0], str)
    if senTestPassed:
        print ("Test generateSentence() passed!")
    else:
        print ("Test generateSentence() failed; did not return a list of strings...")

    print("\n--- TEST: getSentenceLogProbability(...) ---")
    sentences = [sentence, *train_corpus, *test_corpus]
    failed = 0
    for i in range(len(sentences)):
        sen, correct_prob = sentences[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    train_perp = round(model.getCorpusPerplexity(train_corpus), 10)
    test_perp = round(model.getCorpusPerplexity(test_corpus), 10)

    print("Correct train perp.:", trainPerp, '\tYour train perp.:', train_perp, '\t', 'PASSED' if trainPerp == train_perp else 'FAILED')
    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    train_passed, test_passed = train_perp == trainPerp, test_perp == testPerp
    if train_passed and test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on", "the training corpus and the testing corpus..." if not train_passed and not test_passed else "the testing corpus..." if not test_passed else "the training corpus...")

if __name__=='__main__':
    sanityCheck('unigram')

Next, we provide you with another <b>sanity check</b> that trains your model on the *entire* training set, and tests your functions on a small corpus (10 sentences) of *real* test data.

If your code is inefficient, you will likely see that this cell is taking too long. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [ ]:
def sanityCheckFullDataset(model_type):
    model = UnigramModel(train_dataset)
    idxes = list(range(75,7500, 800))
    small_test_corpus = [test_dataset[idx] for idx in idxes]
    if model_type == 'unigram':
        senprobs = [-80.7782190984, -174.4769654449, -136.455148267, -225.5890741503, -719.0142129846, -236.350443633, -126.0056604204, -47.3424655612, -47.7775372096, -138.8159941929]
        testPerp = 881.0132848704
        model = UnigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        senprobs = [-80.8423009715, -174.5131424172, -136.3181234818, -225.357454098, -719.1543898871, -236.6682968913, -126.1965419509, -47.4369338195, -47.7692144935, -138.542462715]
        testPerp = 881.6105352831
        model = SmoothedUnigramModel(train_dataset)
    elif model_type == 'bigram':
        senprobs = [-float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -float('inf'), -32.1502020637, -float('inf'), -float('inf')]
        testPerp = float ('inf')
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-bigram':
        senprobs = [-61.3754065648, -141.9754903887, -107.0849366076, -168.4944718788, -619.9409055374, -195.8159911677, -86.3762008156, -32.4764801981, -48.124714509, -124.687107856]
        testPerp = 261.4247123506
        model = SmoothedBigramModelAD(train_dataset)
    else: assert False, 'Invalid model_type'
    print("\n--- TEST: getSentenceLogProbability(...) ---")
    failed = 0
    for i in range(len(small_test_corpus)):
        sen, correct_prob = small_test_corpus[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    test_perp = round(model.getCorpusPerplexity(small_test_corpus), 10)

    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    test_passed = test_perp == testPerp
    if test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on the testing corpus...")

if __name__=='__main__':
    sanityCheckFullDataset('unigram')

Finally, you can train your model on the full WikiText corpus, and evaluate it on the held-out test set.

In [ ]:
def runModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}
    # Read the corpora
    if model_type == 'unigram':
        model = UnigramModel(train_dataset)
    elif model_type == 'bigram':
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        model = SmoothedUnigramModel(train_dataset)
    else:
        model = SmoothedBigramModelAD(train_dataset)

    print("--------- 5 sentences from your model ---------")
    model.printSentences(5)

    print ("\n--------- Corpus Perplexities ---------")
    print ("Training Set:", model.getCorpusPerplexity(train_dataset))
    print ("Testing Set:", model.getCorpusPerplexity(test_dataset))

if __name__=='__main__':
    runModel('unigram')

## <font color='red'>TODO:</font> Smoothed Unigram Model [25 points]

Here, you will implement each of the 4 functions described above for a <b>unigram</b> model with <b>Laplace (add-one) smoothing</b>. The probability distribution of a word is given by $P_L(w)$. This type of smoothing takes away some of the probability mass for observed events and assigns it to unseen events.

In order to smooth your model, you will need the number of words in the corpus, $N$, and the number of word types, $S$. The distinction between these is meaningful: $N$ indicates the number of word instances, where $S$ refers to the size of our vocabulary. For example, the sentence <em>the cat saw the dog</em> has four word types (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>dog</em>), but five word tokens (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>the</em>, <em>dog</em>). The token <em>the</em> appears twice in the sentence, but they share the same type <em>the</em>.

If $c(w)$ is the frequency of $w$ in the training data, you can compute $P_L(w)$ as follows:

$$P_L(w)=\frac{c(w)+1}{N+S}$$

<font color='green'><b>Hints:</b></font>
* <font color='green'>You may find it convenient to make your `SmoothedUnigramModel` inherit your `UnigramModel`, and then override the function(s) that need to be changed.</font>

In [ ]:
class SmoothedUnigramModel(LanguageModel):
    def __init__(self, trainCorpus):

        ### TODO ###
        pass

    def generateSentence(self):

        ### TODO ###
        return []

    def getSentenceLogProbability(self, sentence):

        ### TODO ###
        return 0.0

    def getCorpusPerplexity(self, testCorpus):

        ### TODO ###
        return 0.0

In [ ]:
if __name__=='__main__':
    sanityCheck('smoothed-unigram')

Since the next sanity check trains your model on the *entire* training set, you will likely see that it is taking too long if you have inefficiences in your code. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [ ]:
if __name__=='__main__':
    sanityCheckFullDataset('smoothed-unigram')

In [ ]:
if __name__=='__main__':
    runModel('smoothed-unigram')

## <font color='red'>TODO:</font> Bigram Model [25 points]

Here, you will implement each of the 4 functions described above for an <b>unsmoothed bigram</b> model. The probability distribution of a word is given by $\hat P(w'|w)$. Thus, the probability of $w_i$ is conditioned on $w_{i-1}$.

<font color='green'><b>Hints:</b></font>
* <font color='green'>You should use a dictionary of dictionaries to store your bigram counts. That is, the outer dictionary should map $w$ to another dictionary that maps $w'$ to the number of times $w'$ occurs after $w$.</font>
* <font color='green'>Do <b>not</b> attempt to iterate over all possible bigrams in your voabulary: <em>only store bigrams that actually occur in your training data.</em> You will run into timeout or out-of-memory issues if you attempt to enumerate all bigrams.</font>
* <font color='green'>Similarly, avoid nested loops over the training data.</font>

In [ ]:
class BigramModel(LanguageModel):
    def __init__(self, trainCorpus):

        ### TODO ###
        pass

    def generateSentence(self):

        ### TODO ###
        return []

    def getSentenceLogProbability(self, sentence):

        ### TODO ###
        return 0.0

    def getCorpusPerplexity(self, testCorpus):

        ### TODO ###
        return 0.0

In [ ]:
if __name__=='__main__':
    sanityCheck('bigram')

Since the next sanity check trains your model on the *entire* training set, you will likely see that it is taking too long if you have inefficiences in your code. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [ ]:
if __name__=='__main__':
    sanityCheckFullDataset('bigram')

In [ ]:
if __name__=='__main__':
    runModel('bigram')

## <font color='red'>TODO:</font> Smoothed Bigram Model [25 points]

Here, you will implement each of the 4 functions described above for a <b>bigram</b> model with <b>absolute discounting</b>. The probability distribution of a word is given by $P_{AD}(w’|w)$.

In order to smooth your model, you need to compute a discounting factor $D$. If $n_k$ is the number of bigrams $w_1w_2$ that appear exactly $k$ times, you can compute $D$ as:

$$D=\frac{n_1}{n_1+2n_2}$$

For each word $w$, you then need to compute the number of bigram types $ww’$ as follows:

$$S(w)=|\{w’\mid c(ww’)>0\}|$$

where $c(ww’)$ is the frequency of $ww’$ in the training data. In other words, $S(w)$ is the number of unique words that follow $w$ at least once in the training data.

Finally, you can compute $P_{AD}(w’|w)$ as follows:

$$P_{AD}(w’|w)=\frac{\max \big (c(ww’)-D,0\big )}{c(w)}+\bigg (\frac{D}{c(w)}\cdot S(w) \cdot P_L(w’)\bigg )$$

where $c(w)$ is the frequency of $w$ in the training data and $P_L(w’)$ is the Laplace-smoothed unigram probability of $w’$.

In [ ]:
class SmoothedBigramModelAD(LanguageModel):
    def __init__(self, trainCorpus):

        ### TODO ###
        pass

    def generateSentence(self):

        ### TODO ###
        return []

    def getSentenceLogProbability(self, sentence):

        ### TODO ###
        return 0.0

    def getCorpusPerplexity(self, testCorpus):

        ### TODO ###
        return 0.0

In [ ]:
if __name__=='__main__':
    sanityCheck('smoothed-bigram')

Since the next sanity check trains your model on the *entire* training set, you will likely see that it is taking too long if you have inefficiences in your code. This cell is expected to run in fewer than <b>10 seconds</b>, so if it takes significantly longer than that, you should probably inspect your code for efficiency issues.

In [ ]:
if __name__=='__main__':
    sanityCheckFullDataset('smoothed-bigram')

In [ ]:
if __name__=='__main__':
    runModel('smoothed-bigram')

## Food for Thought
We provide you some questions to think about. <b>You do not need to answer these questions, but we encourage you to give them some thought.</b>
<ol>
<li>When generating sentences with the unigram model, what controls the length of the generated sentences? How does this differ from the sentences produced by the bigram models?
<li>Consider the probability of the generated sentences according to your models. Do your models assign drastically different probabilities to the different sets of sentences? Why do you think that is?
<li>Look back at the sentences generated using your models. In your opinion, which model produces better / more realistic sentences?
<li>For each of the four models, which test corpus has the highest perplexity? Why?
<li> Why do you think it might be a bad idea to use Laplace (add-one) smoothing for a bigram model? How does the absolute discounting method help?
</ol>

# What to Submit

To submit the assignment, download this notebook as a <TT>.py</TT> file. You can do this by going to <TT>File > Download > Download .py</TT>. Then submit it to the autograder in Gradescope. <b>Do not try to submit it as a <TT>.ipynb</TT> file!</b>

Note that it should take <b>less than 10 minutes</b> to see your score after you have submitted to Gradescope. If your submission runs significantly longer than that, you probably have inefficiency issues in your code!